# imports

In [5]:
import os
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
def get_main_file() -> pd.DataFrame:
    # os.chdir('C:\\Users\\Tural\\OneDrive\\Masaüstü')
    os.chdir('D:\\development\\projects\\ozon-review-scraper\\sample')
    oz_df = pd.read_excel('comments main version.xlsx', sheet_name='Ozon')
    oz_df = oz_df.set_axis(oz_df.iloc[0], axis=1).iloc[2:,][['Link', 'SKU']]
    oz_df = oz_df[~(oz_df['Link'].isna())]
    oz_df = oz_df[~(oz_df['SKU'].duplicated())].reset_index(drop=True)
    return oz_df

# extract base

In [13]:
def xrequest(link_: str, sesh: requests.Session) -> None:
    headers = {
        'Content-Type': 'application/json',
    }
    json_data = {
        'cmd': 'request.get',
        'url': f"{link_}",
        'maxTimeout': 60000,
    }
    response = sesh.post('http://localhost:8191/v1', headers=headers, json=json_data)
    return response
    

In [14]:
url_ = "https://www.ozon.ru/api/entrypoint-api.bx/page/json/v2?url=%2Fproduct%2Fvarochnaya-panel-elektricheskaya-delvento-v30d12s001-30-sm-konforki-hi-light-9-urovney-nagreva-723275899%2F%3Flayout_container%3DpdpPage2column%26layout_page_index%3D2%26sh%3DuDjvkyg2qw"
sesh = requests.Session()
r = xrequest(link_=url_, sesh=sesh)

In [16]:
raw = r.json()['solution']['response']
raw_json = json.loads(BeautifulSoup(raw).find('body').text)

# TODO: CHECK
for k in raw_json['widgetStates'].keys():
    if k.startswith('webAddToCompare'):
        print(k)
    # print(k.split('-')[0])


json.loads(raw_json['widgetStates']['webAddToCompare-939979-pdpPage2column-2'])
# with open('ok1.json', 'w') as f:
#     f.write(json.dumps(raw_json))

webAddToCompare-939979-pdpPage2column-2


{'sku': 723275899,
 'link': '',
 'isInCompare': False,
 'cellTrackingInfo': {'products': {'723275899': '3acc02ed58206d4036a129c6eaa758f4747b7de4'},
  'uis': {'actionAddToCompare': 'a29b0919705734b6c67760695ee4e5a62cdce43a',
   'actionRemoveFromCompare': '3199f50a09628b2bc434ca53062026c5e5b1215b',
   'click': '2824654e401414a4c138e07d47db6176afc46fc1',
   'view': 'c810188488776519aa3c0d70a88724d855a1f4a6'}}}

In [28]:
ok = json.dumps(json.loads(BeautifulSoup(raw).find('body').text))
with open('ok1.json', 'w') as f:
    f.write(ok)

## second request using HTML

In [17]:
url_2 = 'https://www.ozon.ru/product/723275899'
r2 = xrequest(link_=url_2, sesh=sesh)

In [46]:
soup = BeautifulSoup(r2.json()['solution']['response'])

target_script_tag = [script_tag.text for script_tag in soup.find_all('script', {'nonce': ''}) if script_tag.text.startswith('window.__NUXT__')]
assert len(target_script_tag) == 1
target_script_tag = target_script_tag[0]


with open('ok.json', 'wb') as f:
    f.write(target_script_tag.split('window.__NUXT__.')[1][7:-3].encode('utf-8'))



# with open('ok.html', 'w', encoding='utf-8') as f:
#     f.write(r2.json()['solution']['response'])

# extract

In [22]:
def get_content_from_link(link_: str, sku_: str, sesh: requests.Session) -> None:
    headers = {
        'Content-Type': 'application/json',
    }

    json_data = {
        'cmd': 'request.get',
        'url': f"{link_}",
        'maxTimeout': 60000,
    }

    response = sesh.post('http://localhost:8191/v1', headers=headers, json=json_data)
    if response.json()['status'] == 'ok':
        with open(rf'C:\Users\Tural\OneDrive\Masaüstü\ddd\raw_html\{sku_}.html', 'wb') as f:
            f.write(response.json()['solution']['response'].encode('utf-8'))
    else:
        print(f"Not successful for this sku -> {sku_} and link {link_}")
    

In [25]:
for entry_ in get_main_file().to_dict(orient='records'):
    get_content_from_link(link_=entry_['Link'], sku_=entry_['SKU'], sesh=sesh)

In [11]:
sesh = requests.Session()
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'cmd': 'request.get',
    'url': f"{tl}",
    'maxTimeout': 60000,
}

response = sesh.post('http://localhost:8191/v1', headers=headers, json=json_data)

In [21]:
# response.json()['status'] == 'ok'
with open('ok.html', 'wb') as f:
    f.write(response.json()['solution']['response'].encode('utf-8'))

# old version

In [4]:
link_dict = get_main_file().to_dict(orient='records')
os.chdir(r'C:\Users\Tural\OneDrive\Masaüstü\ddd')
for link_dict_entry in link_dict:
    os.system(f'python test_ozon.py {link_dict_entry["Link"]} {link_dict_entry["SKU"]}')


KeyboardInterrupt: 

In [9]:
from bs4 import BeautifulSoup

In [40]:
os.chdir(r'C:\Users\Tural\OneDrive\Masaüstü\ddd\raw_html')
ok = os.listdir()[31]

In [41]:
ok

'V30E02W001.html'

In [42]:
with open(ok, 'rb') as f:
    html_ = f.read()
    
soup = BeautifulSoup(html_)

In [44]:
soup.select_one(SELECTORS['stars'])

In [43]:
sku_data = {
    'sku': ok.rstrip('.html')
}
SELECTORS = {
    'no_data': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(1) > div > div.e0 > div.d0.c7 > div.w6m > h2',
    'stars': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(2) > div > div > div.mr0.mr2.r2m.r4m > div > div.mr0.r2m.r4m > div:nth-child(1) > div > div > div.aa5-a5.aa5-a1.aa5-a8.s6x > div > div.aa5-a6',
    'no_feed': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(2) > div > div > div.mr0.mr2.r2m.r4m > div > div.mr0.r2m.r4m > div:nth-child(1) > div > div > div.x5-a > button > span > span',
    'feed': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(2) > div > div > div.mr0.mr2.r2m.r4m > div > div.mr0.r2m.r4m > div:nth-child(1) > div > div > div.x5-a > a > div > div',
    'price': '#layoutPage > div.b0 > div.container.b4 > div.mr0.m5r > div.mr0.m6r.m3r.rm3 > div.mr0.m6r.m3r.r3m > div > div > div > div.nr3 > div > div > div.m7y.my9.mz0 > div > span.my8.m8y > span',
    'oldprice': '#layoutPage > div.b0 > div.container.b4 > div.mr0.m5r > div.mr0.m6r.m3r.rm3 > div.mr0.m6r.m3r.r3m > div > div > div > div.nr3 > div > div > div.m7y.my9.mz0 > div > span.y8m'
}

sku_data['stars'] = soup.select_one(SELECTORS['stars']).attrs['style']
if no_feed_elem := soup.select_one(SELECTORS['no_feed']):
        sku_data['no_feed'] = no_feed_elem.text
        sku_data['feed'] = ''
else:
    sku_data['no_feed'] = ''
    sku_data['feed'] = soup.select_one(SELECTORS['feed']).text
    
sku_data['price'] = soup.select_one(SELECTORS['price']).text
sku_data['oldprice'] = soup.select_one(SELECTORS['oldprice']).text


sku_data

AttributeError: 'NoneType' object has no attribute 'attrs'

# Parsing RAW HTML

In [27]:
def process_sku(sku_: str, soup) -> None:
    sku_data = {
        'sku': sku_
    }
    SELECTORS = {
        'no_data': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(1) > div > div.e0 > div.d0.c7 > div.w6m > h2',
        'stars': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(2) > div > div > div.mr0.mr2.r2m.r4m > div > div.mr0.r2m.r4m > div:nth-child(1) > div > div > div.aa5-a5.aa5-a1.aa5-a8.s6x > div > div.aa5-a6',
        'no_feed': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(2) > div > div > div.mr0.mr2.r2m.r4m > div > div.mr0.r2m.r4m > div:nth-child(1) > div > div > div.x5-a > button > span > span',
        'feed': '#layoutPage > div.b0 > div.container.b4 > div:nth-child(2) > div > div > div.mr0.mr2.r2m.r4m > div > div.mr0.r2m.r4m > div:nth-child(1) > div > div > div.x5-a > a > div > div',
        'price': '#layoutPage > div.b0 > div.container.b4 > div.mr0.m5r > div.mr0.m6r.m3r.rm3 > div.mr0.m6r.m3r.r3m > div > div > div > div.nr3 > div > div > div.m7y.my9.mz0 > div > span.my8.m8y > span',
        'oldprice': '#layoutPage > div.b0 > div.container.b4 > div.mr0.m5r > div.mr0.m6r.m3r.rm3 > div.mr0.m6r.m3r.r3m > div > div > div > div.nr3 > div > div > div.m7y.my9.mz0 > div > span.y8m'
    }

    if soup.select_one(SELECTORS['stars']):
        sku_data['stars'] = soup.select_one(SELECTORS['stars']).attrs['style']
        if no_feed_elem := soup.select_one(SELECTORS['no_feed']):
                sku_data['no_feed'] = no_feed_elem.text
                sku_data['feed'] = ''
        else:
            sku_data['no_feed'] = ''
            sku_data['feed'] = soup.select_one(SELECTORS['feed']).text

        sku_data['price'] = soup.select_one(SELECTORS['price']).text
        sku_data['oldprice'] = soup.select_one(SELECTORS['oldprice']).text
        return sku_data
    else:
        sku_data['no_feed'] = ''
        sku_data['stars'] = 'width:%;'
        sku_data['feed'] = ''
        sku_data['price'] = ''
        sku_data['oldprice'] = ''
        
        return sku_data

In [30]:
os.chdir(r'C:\Users\Tural\OneDrive\Masaüstü\ddd\raw_html')
# ok = os.listdir()[31]
ready_skus = []

for i in os.listdir():
    with open(i, 'rb') as f:
        html_ = f.read()
    soup_ = BeautifulSoup(html_)
    sku_ = i.rstrip('.html')
    ready_skus.append(process_sku(sku_=sku_, soup=soup_))


    


In [31]:
def check_feeds(tp):
    if (tp[0] == '') and (tp[1] == ''):
        return 0
    elif (tp[0] == ''):
        return int(tp[1].split('\xa0')[0])
    else:
        return 0

out_df = pd.DataFrame(ready_skus).reset_index(drop=True)
out_df['stars'] = out_df['stars'].map(
                        lambda x: x.split(':')[1].split('%')[0]
                    ).map(
                        lambda x: round(float(x), 2) if x != '' else 0.0
                    ).map(
                        lambda x: round((x / 100.0) * 5.0, 1)
                    )
out_df['feeds'] = out_df[['no_feed', 'feed']].apply(tuple, axis=1).map(check_feeds)
out_df['price'] = out_df['price'].map(
                lambda x: int(x.replace('\u2009', '').replace('₽', ''))
                if x != '' else 0
            )
out_df['oldprice'] = out_df['oldprice'].map(
                lambda x: int(x.replace('\u2009', '').replace('₽', ''))
                if x != '' else 0
            )
out_df

,sku,no_feed,stars,feed,price,oldprice,feeds
0,D2501,,0.0,,0,0,0
1,D2502,,0.0,,0,0,0
2,D2504,,0.0,,0,0,0
3,D2505,,0.0,,0,0,0
4,D2506,,0.0,,0,0,0
...,...,...,...,...,...,...,...
109,VWB4701,,0.0,,0,0,0
110,VWB4702,,0.0,,0,0,0
111,VWB4900,,0.0,,0,0,0
112,VWB6701,,0.0,,0,0,0


In [32]:
out_df.to_excel(r'C:\Users\Tural\OneDrive\Masaüstü\ddd\output\ozon.xlsx')